In [1]:
import math
import pandas as pd
from langdetect import detect
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer
import math
import matplotlib.pyplot as plt

In [2]:
lem = WordNetLemmatizer() #create lemmatizer

In [3]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shirinharandi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
dictionary = pd.read_csv('../data/processedDict.csv')
dictionary['word'] = dictionary['word'].apply(lambda x: lem.lemmatize(x, pos='n'))

In [ ]:
# DELETE LATER
# mask = (reviews['date'] >= '2013-01-01') & (reviews['date'] < '2014-01-01')
# reviews = reviews.loc[mask].copy()
# reviews[:1000]

In [80]:
def get_trends_nice_og(reviews):
    
    def get_unique_words(rev):
        
        allwords = rev['comments'].tolist()
        allwords = " ".join(allwords)
        unique_words = set(allwords.split(' '))

        unique_words = list(unique_words)
        unique_words = [string for string in unique_words if string != ""]

        ls = []
        for word in unique_words:
            word = ''.join([i for i in word if not i.isdigit()])
            ls += [word]
        unique_words= ls
        unique_words = [string for string in unique_words if string != ""]
        unique_words = list(dict.fromkeys(unique_words))
        return unique_words
    
    def get_masked_reviews(start_year, end_year):
        mask = (reviews['date'] >= start_year) & (reviews['date'] < end_year)
        return reviews.loc[mask].copy()
    
#     def get_reviews_list(rev):
        
    
    old_reviews =  get_masked_reviews('2013-01-01', '2015-01-01')
#     print(old_reviews)
    new_reviews =  get_masked_reviews('2016-01-01', '2018-01-01')

    start_year_words = get_unique_words(old_reviews)
    end_year_words = get_unique_words(new_reviews)
    
    old_reviews_list = old_reviews['comments'].tolist()
    new_reviews_list = new_reviews['comments'].tolist()

    print(new_reviews_list)
    
    def get_freq(word, rev):
        total = 0.0
        rev = list(rev)
        for r in rev:
            total += math.log(1 + r.count(word))
        return total

    column_names = ["word", "old_frequency", "new_frequency"]
    out = pd.DataFrame(columns = column_names)
    
    old_total = 0.0
    for word in start_year_words:
        f = get_freq(word, old_reviews_list)
        old_total += f
        out = out.append({'word': word, 'old_frequency': f, "new_frequency": 0.0}, ignore_index=True)
        
    new_total = 0.0
    for word in end_year_words:
        f = get_freq(word, new_reviews_list)
        new_total += f
        if word in start_year_words:
            ind = out.index[out['word']==word]
            out.loc[ind,"new_frequency"] = f
        else:
            out = out.append({'word': word, 'old_frequency': 0.0, 'new_frequency': f}, ignore_index=True)
    
    out["old_sum_count"] = old_total
    out["new_sum_count"] = new_total
    return out

In [60]:
def get_trends_nice(reviews):
    
    def get_unique_words(rev):
        
        allwords = rev['comments'].tolist()
        allwords = " ".join(allwords)
        unique_words = set(allwords.split(' '))

        unique_words = list(unique_words)
        unique_words = [string for string in unique_words if string != ""]

        ls = []
        for word in unique_words:
            word = ''.join([i for i in word if not i.isdigit()])
            ls += [word]
        unique_words= ls
        unique_words = [string for string in unique_words if string != ""]
        unique_words = list(dict.fromkeys(unique_words))
        return unique_words
    
    def get_masked_reviews(start_year, end_year):
        mask = (reviews['date'] >= start_year) & (reviews['date'] < end_year)
        return reviews.loc[mask].copy()
    
#     def get_reviews_list(rev):
        
    
    old_reviews =  get_masked_reviews('2013-01-01', '2014-01-01')
#     print(old_reviews)
    new_reviews =  get_masked_reviews('2017-01-01', '2018-01-01')

    start_year_words = get_unique_words(old_reviews)
    end_year_words = get_unique_words(new_reviews)
    
    in_first = set(start_year_words)
    in_second = set(end_year_words)

    in_second_but_not_in_first = in_second - in_first

    total_words_in_reviews = start_year_words + list(in_second_but_not_in_first)
    
    old_reviews_list = old_reviews['comments'].tolist()
    new_reviews_list = new_reviews['comments'].tolist()


    def get_freq(word, rev):
        total = 0.0
        rev = list(rev)
        for r in rev:
            total += math.log(1 + r.count(word))
        return total

    column_names = ["word", "old_frequency", "new_frequency"]
    out = pd.DataFrame(columns = column_names)
    
    old_total = 0.0
    new_total = 0.0
    for word in total_words_in_reviews:
        f = get_freq(word, old_reviews_list)
        old_total += f
#         print(f)
        f2 = get_freq(word, new_reviews_list)
        new_total += f2
        out = out.append({'word': word, 'old_frequency': f, "new_frequency": f2}, ignore_index=True)
        
    out["old_sum_count"] = old_total
    out["new_sum_count"] = new_total
    return out

In [ ]:
# filepath = '/Users/shirinharandi/Desktop/COMP0031/Data/en_reviews/manchester_en.csv'
# city_name = "Manchester"
# r = pd.read_csv(filepath)

# # reviews = reviews['date']
# # reviews

# table = str.maketrans('', '', string.punctuation) #mapping to strip punctuation in review

# #strip punct of each review -> lemmatise -> output is list of words so join into sentences
# r['comments'] = r.comments.apply(lambda review: ' '.join(map(str, [lem.lemmatize(word.translate(table), pos='n') for word in review.lower().split()])))


# r['date'] = pd.to_datetime(r['date'])

# soci = get_trends_nice_og(r)
# soci

In [7]:
dictionary_words = dictionary['word'].tolist()
dictionary_words = list(set(dictionary_words))
# dictionary_words

In [81]:
# cities = ["tokyo", "belize", "santiago", "taipei", "singapore", "beijing", "hong_kong", "mexico_city", "buenos_aires", "rio"]
cities = ["manchester"]

column_names = ["word", "old_frequency", "new_frequency","old_sum_count", "new_sum_count","city", "Ofrequency", "Nfrequency","freq", "gain", "licia_gain"]
big_out = pd.DataFrame(columns = column_names)

for c in cities: 
    path = "/Users/shirinharandi/Desktop/COMP0031/Data/en_reviews/{}_en.csv".format(c)
    filepath = path
    city_name = c
    reviews = pd.read_csv(filepath)

    table = str.maketrans('', '', string.punctuation) #mapping to strip punctuation in review

    #strip punct of each review -> lemmatise -> output is list of words so join into sentences
    reviews['comments'] = reviews.comments.apply(lambda review: ' '.join(map(str, [lem.lemmatize(word.translate(table), pos='n') for word in review.lower().split()])))
    reviews['date'] = pd.to_datetime(reviews['date'])
    
    out = get_trends_nice_og(reviews)
    
    test = out[out['word'].isin(dictionary_words)]

    test['city'] = city_name
    test['Ofrequency'] = test['old_frequency'] / test['old_sum_count']
    test['Nfrequency'] = test['new_frequency'] / test['new_sum_count']
    test['freq'] = (test['Nfrequency'] + test['Ofrequency'])/2
    test['gain'] = np.where(test['Ofrequency'] == 0, 1000 ,test['Nfrequency'] / test['Ofrequency'])
    test['licia_gain'] = np.where(test['Ofrequency'] == 0, 1000 ,test['new_frequency'] / test['old_frequency'])
    
#     print(test)
    big_out = pd.concat([big_out, test], ignore_index=True) 


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [82]:
# big_out.to_csv(r'~/Downloads/big_out.csv', index = False)
big_out

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain
0,daughter,17.734145,98.898032,233387.119852,4.054021e+06,manchester,0.000076,2.439505e-05,5.019050e-05,0.321047,5.576702
1,convenient,79.542027,943.348299,233387.119852,4.054021e+06,manchester,0.000341,2.326945e-04,2.867552e-04,0.682757,11.859747
2,text,9.704061,68.163990,233387.119852,4.054021e+06,manchester,0.000042,1.681392e-05,2.919658e-05,0.404383,7.024275
3,microwave,1.386294,55.569557,233387.119852,4.054021e+06,manchester,0.000006,1.370727e-05,9.823580e-06,2.307663,40.084963
4,advice,29.112182,201.130465,233387.119852,4.054021e+06,manchester,0.000125,4.961258e-05,8.717516e-05,0.397735,6.908808
...,...,...,...,...,...,...,...,...,...,...,...
315,sympathy,0.000000,0.693147,233387.119852,4.054021e+06,manchester,0.000000,1.709777e-07,8.548885e-08,1000.000000,1000.000000
316,roof,0.000000,44.479203,233387.119852,4.054021e+06,manchester,0.000000,1.097163e-05,5.485813e-06,1000.000000,1000.000000
317,transit,0.000000,27.438205,233387.119852,4.054021e+06,manchester,0.000000,6.768145e-06,3.384073e-06,1000.000000,1000.000000
318,responds,0.000000,25.646446,233387.119852,4.054021e+06,manchester,0.000000,6.326175e-06,3.163087e-06,1000.000000,1000.000000


In [83]:
big_out[big_out['word'] == "easygoing"]

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain
154,easygoing,9.010913,40.895684,233387.119852,4.054021e+06,manchester,0.000039,0.00001,0.000024,0.261276,4.538462


In [63]:
column_names = ["word", "gain_mean"]
mean_gain = pd.DataFrame(columns = column_names)

for dic in dictionary_words:
    dic_rows = big_out[big_out['word'] == dic]
    dic_rows = dic_rows[dic_rows['gain'] != 1000]
    f = dic_rows['gain'].mean()
    mean_gain = mean_gain.append({'word': dic, 'gain_mean': f}, ignore_index=True)

mean_gain

,word,gain_mean
0,easygoing,0.193727
1,furniture,NaN
2,attraction,0.516605
3,equipped,1.333730
4,wife,1.008843
...,...,...
316,size,2.853919
317,family,0.972220
318,response,2.970479
319,blanket,0.453249


In [64]:
top = mean_gain.nlargest(20, 'gain_mean')
bottom = mean_gain.nsmallest(20, 'gain_mean')
li = pd.concat([top, bottom], ignore_index=True)
li

,word,gain_mean
0,fridge,9.761892
1,suggestion,5.639604
2,reply,4.649445
3,communication,4.599491
4,step,3.810406
5,drive,3.610560
6,check,3.573420
7,promptly,3.573184
8,exactly,3.406399
9,communicating,3.357932


In [65]:
main_rows = pd.DataFrame()
for w in li['word']:
#     print(w)
    temp = big_out[big_out['word'] == w]
    temp = temp[temp['gain'] != 1000]
    main_rows = pd.concat([main_rows, temp], ignore_index=True) 

main_rows

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain
0,fridge,0.693147,157.174511,108600.183515,2.522628e+06,manchester,0.000006,0.000062,0.000034,9.761892,226.754888
1,suggestion,0.693147,90.802281,108600.183515,2.522628e+06,manchester,0.000006,0.000036,0.000021,5.639604,131.000000
2,reply,0.693147,74.859896,108600.183515,2.522628e+06,manchester,0.000006,0.000030,0.000018,4.649445,108.000000
3,communication,9.704061,1036.778283,108600.183515,2.522628e+06,manchester,0.000089,0.000411,0.000250,4.599491,106.839635
4,step,2.079442,184.052032,108600.183515,2.522628e+06,manchester,0.000019,0.000073,0.000046,3.810406,88.510318
5,drive,2.079442,174.398958,108600.183515,2.522628e+06,manchester,0.000019,0.000069,0.000044,3.610560,83.868171
6,check,15.772486,1309.202506,108600.183515,2.522628e+06,manchester,0.000145,0.000519,0.000332,3.573420,83.005463
7,promptly,0.693147,57.531216,108600.183515,2.522628e+06,manchester,0.000006,0.000023,0.000015,3.573184,83.000000
8,exactly,4.158883,329.075012,108600.183515,2.522628e+06,manchester,0.000038,0.000130,0.000084,3.406399,79.125815
9,communicating,0.693147,54.065480,108600.183515,2.522628e+06,manchester,0.000006,0.000021,0.000014,3.357932,78.000000


In [66]:
main_rows.to_csv(r'~/Downloads/main_rows.csv', index = False)

In [67]:
def get_cat_lev1(word):
    ind = dictionary.index[dictionary['word']==word]
    return dictionary.loc[ind, 'cat_lev1'].tolist()[0]

main_rows['cat'] = main_rows['word'].apply(lambda x: get_cat_lev1(x))
main_rows

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain,cat
0,fridge,0.693147,157.174511,108600.183515,2.522628e+06,manchester,0.000006,0.000062,0.000034,9.761892,226.754888,business
1,suggestion,0.693147,90.802281,108600.183515,2.522628e+06,manchester,0.000006,0.000036,0.000021,5.639604,131.000000,business
2,reply,0.693147,74.859896,108600.183515,2.522628e+06,manchester,0.000006,0.000030,0.000018,4.649445,108.000000,business
3,communication,9.704061,1036.778283,108600.183515,2.522628e+06,manchester,0.000089,0.000411,0.000250,4.599491,106.839635,business
4,step,2.079442,184.052032,108600.183515,2.522628e+06,manchester,0.000019,0.000073,0.000046,3.810406,88.510318,business
5,drive,2.079442,174.398958,108600.183515,2.522628e+06,manchester,0.000019,0.000069,0.000044,3.610560,83.868171,business
6,check,15.772486,1309.202506,108600.183515,2.522628e+06,manchester,0.000145,0.000519,0.000332,3.573420,83.005463,business
7,promptly,0.693147,57.531216,108600.183515,2.522628e+06,manchester,0.000006,0.000023,0.000015,3.573184,83.000000,business
8,exactly,4.158883,329.075012,108600.183515,2.522628e+06,manchester,0.000038,0.000130,0.000084,3.406399,79.125815,business
9,communicating,0.693147,54.065480,108600.183515,2.522628e+06,manchester,0.000006,0.000021,0.000014,3.357932,78.000000,business


In [68]:
main_rows.to_csv(r'~/Downloads/main_rows2.csv', index = False)

In [69]:
top['temp'] =  top['word'].apply(lambda x: get_cat_lev1(x))

In [70]:
top

,word,gain_mean,temp
271,fridge,9.761892,business
243,suggestion,5.639604,business
36,reply,4.649445,business
162,communication,4.599491,business
247,step,3.810406,business
56,drive,3.610560,business
250,check,3.573420,business
315,promptly,3.573184,business
265,exactly,3.406399,business
30,communicating,3.357932,business


In [71]:
bottom['temp'] =  bottom['word'].apply(lambda x: get_cat_lev1(x))
bottom

,word,gain_mean,temp
142,tube,0.107626,business
153,dinner,0.133416,social
79,delightful,0.135824,social
155,backyard,0.143501,business
93,company,0.147646,social
158,subway,0.172202,business
226,historic,0.180596,business
292,delicious,0.186376,social
0,easygoing,0.193727,social
282,them,0.205850,social


In [72]:
testing = big_out

In [76]:
testing

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain
0,easygoing,4.158883,18.714974,108600.183515,2.522628e+06,manchester,0.000038,0.000007,2.285710e-05,0.193727,4.500000
1,daughter,8.030084,45.577815,108600.183515,2.522628e+06,manchester,0.000074,0.000018,4.600465e-05,0.244349,5.675883
2,flight,4.564348,150.426489,108600.183515,2.522628e+06,manchester,0.000042,0.000060,5.082989e-05,1.418806,32.956839
3,charming,10.109526,62.383246,108600.183515,2.522628e+06,manchester,0.000093,0.000025,5.890943e-05,0.265653,6.170739
4,shes,4.158883,39.051808,108600.183515,2.522628e+06,manchester,0.000038,0.000015,2.688798e-05,0.404242,9.389975
...,...,...,...,...,...,...,...,...,...,...,...
314,concern,0.693147,26.745058,108600.183515,2.522628e+06,manchester,0.000006,0.000011,8.492311e-06,1.661099,38.584963
315,accomodating,0.000000,44.361420,108600.183515,2.522628e+06,manchester,0.000000,0.000018,8.792699e-06,1000.000000,1000.000000
316,pictured,0.000000,20.101268,108600.183515,2.522628e+06,manchester,0.000000,0.000008,3.984192e-06,1000.000000,1000.000000
317,terrace,0.693147,35.755971,108600.183515,2.522628e+06,manchester,0.000006,0.000014,1.027833e-05,2.220754,51.584963
